In [22]:
import asyncio
from pprint import pprint

from pydantic import BaseModel
from agents import Agent, Runner, InputGuardrail, \
                   GuardrailFunctionOutput, trace
from agents.exceptions import InputGuardrailTripwireTriggered

from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

# 1.0 Introduction

### 1.1 Basic Concepts

To get started, you'll need to be aware of 3 very important [primitives]():

> **Agents**, which are LLMs equipped with instructions and tools.  
> **Handoffs**, which allow agents to delegate to other agents for specific tasks.  
> **Guardrails**, which enable the inputs to agents to be validated.  

With that in mind, we'll generally do 3 steps when calling our Agents:
1. Creating an `Agent` instance.
2. Using `with trace()` to track the Agent.
3. Calling `runner.run()` to run the Agent.

Additionally:
- For tracking our Agent's interactions & output we can go to: https://platform.openai.com/traces
- For documentation, we can go to: https://openai.github.io/openai-agents-python/

In [3]:
agent = Agent(
    name="Jokester",
    instructions="You are a comedian. You are going to tell jokes.",
    model="gpt-4o-mini",
)

In [7]:
with trace("Testing-2025-08-23"):
    result = await Runner.run(agent, "Tell me a joke about Ronaldo vs. Messi.")
    print(result.final_output)

Why don’t Ronaldo and Messi ever play hide and seek?

Because good luck hiding when Messi can dribble past you, and Ronaldo will just take a selfie from wherever he is!


In [8]:
result

RunResult(input='Tell me a joke about Ronaldo vs. Messi.', new_items=[MessageOutputItem(agent=Agent(name='Jokester', instructions='You are a comedian. You are going to tell jokes.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_68a9c87aa8e0819f809b3e4988ec771e0b13905296926762', content=[ResponseOutputText(annotations=[], text='Why don’t Ronaldo and Messi ever play hide and seek?\n\nBecause good luck hiding when Messi can dribble past you, and Ronaldo will just take 

In [9]:
print(result)

RunResult:
- Last agent: Agent(name="Jokester", ...)
- Final output (str):
    Why don’t Ronaldo and Messi ever play hide and seek?
    
    Because good luck hiding when Messi can dribble past you, and Ronaldo will just take a selfie from wherever he is!
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### 1.2 Using multiple agents with handoffs

In [12]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

In [13]:
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

In [14]:
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

In [15]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

In [23]:
pprint(history_tutor_agent)

Agent(name='History Tutor',
      instructions='You provide assistance with historical queries. Explain '
                   'important events and context clearly.',
      handoff_description='Specialist agent for historical questions',
      handoffs=[],
      model=None,
      model_settings=ModelSettings(temperature=None,
                                   top_p=None,
                                   frequency_penalty=None,
                                   presence_penalty=None,
                                   tool_choice=None,
                                   parallel_tool_calls=None,
                                   truncation=None,
                                   max_tokens=None,
                                   reasoning=None,
                                   metadata=None,
                                   store=None,
                                   include_usage=None,
                                   extra_query=None,
                                   

In [16]:
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

In [17]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

In [18]:
async def main():
    # Example 1: History question
    try:
        result = await Runner.run(triage_agent, "who was the first president of the united states?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

    # Example 2: General/philosophical question
    try:
        result = await Runner.run(triage_agent, "What is the meaning of life?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

In [ ]:
# if we ran this via a Python script, use `asyncio.run(main())`
await main()

The first President of the United States was George Washington. He served from April 30, 1789, to March 4, 1797. Washington is known for leading the Continental Army to victory over the British in the American Revolutionary War and for presiding over the Constitutional Convention of 1787, which drafted the United States Constitution. His presidency set many precedents, including the formation of a Cabinet and the tradition of a two-term limit, which later became law with the 22nd Amendment. Washington is often referred to as the "Father of His Country" for his pivotal role in founding the United States.
Guardrail blocked this input: Guardrail InputGuardrail triggered tripwire
